# Designing 3D CNNs

The dimension of a CNN does not refer to the dimensions of the input but to the dimensions of the kernel stride.  
1D kernel moves only left-right (or up-down)  
2D kernel moves left-right and up-down  
3D kernel moves left-right, up-down and forward-backwards.   

Thus with a kernel of size (3,3,20) a 3D volume of size (150,150,20) could be processed. The present 2D CNN from pytorch and fastai could thus easily be adapted. However, small findings which only occur in a feq slices could disappear in the convolutions, so 3D CNNs with smaller kernels might be better.  



In [5]:
# default_exp models
# export 

import torchvision
from torch import nn

## Custom Modules

In [3]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """
    def __init__(self, sigma=0.1, is_relative_detach=True):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.register_buffer('noise', torch.tensor(0))

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.expand(*x.size()).float().normal_() * scale
            x = x + sampled_noise
        return x 

Somehow, in 3D CNNs, the input is not transfered to cuda. I believe something in the transforms is wrong. Until this is fixed, subcalssing nn.Sequential is the workarround

In [ ]:
# export
class Sequential_(nn.Sequential):
    "Similar to nn.Sequential, but copies input to cuda"
    def forward(self, input):
        for module in self:
            input = module(input.cuda())
        return input

## Custom 3D Networks

In [4]:
# export
def resnet_3d(n_input, n_classes):
    return Sequential_(
        # 1st Conv Block
        nn.Conv3d(n_input, 128, kernel_size = (7,7,3), stride = (2, 2, 1), padding = (3, 3, 1), bias = True),
        nn.BatchNorm3d(128, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
        nn.ReLU(),
        nn.Dropout3d(),

        # 2nd Conv Block
        nn.Conv3d(128, 256, kernel_size = (4,4,3), stride = (2, 2, 1), padding = (1, 1, 1), bias = True),
        nn.BatchNorm3d(256, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
        nn.ReLU(),
        nn.Dropout3d(),
        
        # 3rd Conv Block
        nn.Conv3d(256, 384, kernel_size = (1,1,3), stride = (1, 1, 2), padding = (0, 0, 0), bias = True),
        nn.BatchNorm3d(384, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
        nn.ReLU(),
        nn.Dropout3d(),
        
        # 1st Res Block
        nn.Conv3d(384, 512, kernel_size = (3,3,3), stride = (1, 1, 1), padding = (1, 1, 1), bias = True),
        nn.BatchNorm3d(512, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
        nn.ReLU(),
        
        nn.Conv3d(512, 512, kernel_size = (3,3,3), stride = (1, 1, 1), padding = (1, 1, 1), bias = True),
        nn.BatchNorm3d(512, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
        nn.ReLU(),
        
        nn.Conv3d(512, 512, kernel_size = (3,3,3), stride = (1, 1, 1), padding = (1, 1, 1), bias = True),
        nn.BatchNorm3d(512, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
        nn.ReLU(),
        
        nn.Conv3d(512, 512, kernel_size = (3,3,3), stride = (1, 1, 1), padding = (1, 1, 1), bias = True),
        nn.BatchNorm3d(512, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
        nn.ReLU(),
        nn.Dropout3d(),
                
        nn.AdaptiveAvgPool3d(1),
        nn.Flatten(),
        nn.Linear(512, n_classes), 
        nn.Softmax(dim = 1))

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(20, 128, kernel_size=11, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.AvgPool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 2)
        self.softmax =  nn.LogSoftmax(dim=1)
        self.flatten = Flatten()

    def forward(self, x):
        if x.device.type == 'cpu': x = x.cuda()
        out = self.layer1(x)
   #     print(out.shape)
        out = self.layer2(out)
   #     print(out.shape)
        out = self.AvgPool(out)
  #      print(out.shape)
        out = self.flatten(out)
  #      print(out.shape)
        out = self.fc1(out)
 #       print(out.shape)
        out = self.fc2(out)
        out = self.softmax(out)
        # print(out.shape)
        return out